In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

In [5]:
url = 'https://www.dictionary.com/browse/'
word = 'blithe'

# Cert locale: '/Users/ericschlosser/Desktop/TheHardWay/GRE_Scraper_Support/dictionary.com.cer'
# 'css-irpbha e7pf46o0'
results = requests.get(url + word, verify = False)
src = results.content
soup = bs(src, 'lxml')

text = soup.select('.luna-pos,.one-click-content')

for tag in text:
    print(tag.text.strip())
    
#print(soup.prettify())

/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


adjective,
without thought or regard; carefree; heedless: a blithe indifference to anyone's feelings.
joyous, merry, or happy in disposition; glad; cheerful: Everyone loved her for her blithe spirit.
2 happy,  mirthful,  sprightly,  lighthearted,  buoyant,  joyful,  blithesome.
2 joyless.
First recorded before 1000; Middle English; Old English blÄ«the;  cognate with Old Norse blÄ«thr, Old High German blÄ«di, Gothic bleiths
blitheÂ·ful, adjective
adjective
blitheÂ·fulÂ·ly, adverb
adverb
blitheÂ·ly, adverb
adverb
blitheÂ·ness, noun
noun
oÂ·verÂ·blithe, adjective
adjective
noun
a female given name.
Sadly, Republicansâ€”who have repeatedly slammed Obama for this kind of blithe incoherenceâ€”are not immune to the same disorder.
I missed Don's chiseled mug and Roger's blithe wisecracks and Peggy's prickly chutzpah.
Flashing a smug grin while throwing up your collective shoulders in blithe befuddlement should convince absolutely no one.
But by all accounts he was a rather blithe spirit: witty

In [ ]:
# def: one-click-content css-nnyc96 e1q3nk1v1
# word level: css-irpbha e7pf46o0

In [ ]:
text = soup.find_all(class_ = ['css-irpbha e7pf46o0','one-click-content css-nnyc96 e1q3nk1v1'])
df_tdlist = pd.DataFrame(columns = ['Words', 'Definitions'])
#df_tdlist['Words'] = ['space']
#df_tdlist['Definitions'] = ['space']
word_list = ['ape', 'blithe', 'turncoat']

In [ ]:
for tag in text:
    print(tag)

In [ ]:
for tag in text:
    print(tag.text.strip())

In [ ]:
def definitions(word_list):
    df_word = pd.DataFrame(columns = ['Words', 'Parts of Speech', 'Definitions'])
    url = 'https://www.dictionary.com/browse/'
    
    for word in word_list:
        def_list = []
        pos_list = []
        results = requests.get(url + word, verify = False)
        src = results.content
        soup = bs(src, 'lxml')
        text = soup.find_all(class_ = ['luna-pos', 'one-click-content'])
        
        for tag in text:
            if tag.text.strip() in ['noun', 'verb', 'adverb', 'adjective']:
                pos = tag.text.strip()
            #Dealing with pos exceptions
            elif tag.text.strip()[:4] == 'verb':
                pos = 'verb'
            else:
                defi = tag.text.strip()
                df_word['Definitions'] = [defi]
                df_word['Parts of Speech'] = [pos]
                df_word['Words'] = [word]
                try:
                    #print(df_word)
                    df_tdlist = pd.concat([df_tdlist, df_word], axis = 0, ignore_index = True)
                except:
                    df_tdlist = df_word
    
    df_tdlist.drop_duplicates(inplace = True)
    # Eliminates 'Disparaging and Offensive.' entries
    m1 = (df_tdlist['Definitions'] == df_tdlist['Definitions'].shift(1))
    m2 = df_tdlist['Definitions'].shift(1) == 'Disparaging and Offensive.'
    df_tdlist = df_tdlist[~m2]
    # Eliminates word senses
    df_tdlist  = df_tdlist[df_tdlist['Definitions'].str[0].str.islower()]

    print(df_tdlist)
    return df_tdlist
   

In [ ]:
word_list = ['blithe']
df_tdlist = definitions(word_list)

In [ ]:


print(df_tdlist)

Dealing with 'sepcifically' in definitions

In [ ]:
url = 'https://www.merriam-webster.com/dictionary/'
word = 'turncoat'
results = requests.get(url + word)
src = results.content
soup = bs(src, 'lxml')
text = soup.find_all(class_= ['num', 'letter', 'dtText', 'sdsense', 'important-blue-link'])

pos_list = []
def_list = []
num = ''
let = ''
numletprev = ''

for tag in text:
    if tag.text.strip() in '1234567890':
        num = tag.text.strip()
    elif tag.text.strip() in 'abcdefghij':
        let = tag.text.strip() 
    elif tag.text.strip() in ['noun', 'verb', 'adverb', 'adjective']:
        pos = tag.text.strip()
        if pos in pos_list:
            break
        pos_list.append(pos)
    else:
        if num+let == numletprev:
            num = ''
            let = '  '
        defi = tag.text.strip()
        if defi[:10] == 'especially':
            esp = defi.split('\n')
            def_list[-1] = def_list[-1] + ' ' + esp[0]+ ':' +esp[1][1:]
        if defi[:12] == 'specifically':
            esp = defi.split('\n')
            def_list[-1] = def_list[-1] + ' ' + esp[0]+ ":" +esp[1][1:]

        else:
            def_list.append(str(pos)+ ': ' + defi[2:])
            numletprev = num+let
            
            
            
             
print(def_list)

In [ ]:
url = 'https://www.merriam-webster.com/dictionary/'
word = 'ape'
results = requests.get(url + word)
src = results.content
soup = bs(src, 'lxml')
text = soup.find_all(class_= ['sb-0', 'important-blue-link'])

pos_list = []
def_list = []
num = ''
let = ''
numletprev = ''

for tag in text:
    if tag.text.strip() in ['noun', 'verb', 'adjective', 'adverb', 'pronoun']:
        pos = tag.text.strip()
    elif tag.text.strip() in ['transitive verb']:
        pass
    else:
        def_list.append(tag.text.strip())
            
for part in def_list:        
    print(part.split('\n'))       


In [ ]:
def definitions(word_list):
    url = 'https://www.merriam-webster.com/dictionary/'
    
    num = ''
    let = ''
    
    numletprev = ''
    
    for word in word_list:
        def_list = []
        pos_list = []
        results = requests.get(url + word)
        src = results.content
        soup = bs(src, 'lxml')
        text = soup.find_all(class_= ['num', 'letter', 'dtText', 'sdsense', 'important-blue-link'])
        
        for tag in text:
            if tag.text.strip() in '1234567890':
                num = tag.text.strip()
            elif tag.text.strip() in 'abcdefghij':
                let = tag.text.strip() 
            elif tag.text.strip() in ['noun', 'verb', 'adverb', 'adjective']:
                pos = tag.text.strip()
                if pos in pos_list:
                    break
                pos_list.append(pos)
            else:
                if num+let == numletprev:
                    num = ''
                    let = '  '
                defi = tag.text.strip()
                if defi[:10] == 'especially':
                    esp = defi.split('\n')
                    print(esp)
                    def_list[-1] = def_list[-1] + ' ' + esp[0]+ ':' +esp[1][1:]
                elif defi[:12] == 'specifically':
                    esp = defi.split('\n')
                    print(esp)
                    def_list[-1] = def_list[-1] + ' ' + esp[0]+ ":" +esp[1][1:]

                else:
                    def_list.append(str(pos)+ ': ' + defi[2:])
                    numletprev = num+let
                    
        df_word = pd.DataFrame(columns = ['Words', 'Definitions'])
        df_word['Definitions'] = def_list
        df_word['Words'] = [word]*len(def_list)
        try:
            #print(df_word)
            df_tdlist = pd.concat([df_tdlist, df_word], axis = 0, ignore_index = True)
        except:
            df_tdlist = df_word
    print(df_tdlist)
    return df_tdlist.iat[11,1]
    

In [ ]:
definitions(word_list)

In [ ]:
df_tdlist.iat[11,1]

In [ ]:
import pandas as pd
data = pd.read_csv('ner_dataset 2.csv', encoding= 'unicode_escape')
data.head()

In [ ]:
data.shape


In [ ]:
from itertools import chain
def get_dict_map(data, token_or_tag):
    tok2idx = {}
    idx2tok = {}
    
    if token_or_tag == 'token':
        vocab = list(set(data['Word'].to_list()))
    else:
        vocab = list(set(data['Tag'].to_list()))
    
    idx2tok = {idx:tok for  idx, tok in enumerate(vocab)}
    tok2idx = {tok:idx for  idx, tok in enumerate(vocab)}
    return tok2idx, idx2tok


token2idx, idx2token = get_dict_map(data, 'token')
tag2idx, idx2tag = get_dict_map(data, 'tag')